# 기계학습개론 과제3
응용통계학과 </p>
32197453 이동현

## 데이터 불러오기 및 전처리

In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from xgboost import plot_importance, XGBClassifier
from lightgbm import LGBMClassifier, plot_importance
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, f1_score, precision_recall_curve
from sklearn.metrics import roc_curve, average_precision_score, matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier

**평가지표 출력함수 생성**

In [73]:
def metrics(y_test, y_pred):
    confm = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    AUC = roc_auc_score(y_test, y_pred)
    AUPRC = average_precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    MCC = matthews_corrcoef(y_test, y_pred)
    print('Confusion_Matrix: \n {0} \n Accuracy : {1:.3f} \n AUC : {2:.3f} \n AUPRC : {3:.3f} \n F1-score : {4:.3f} \n MCC : {5:.3f}'
        .format(confm,accuracy, AUC, AUPRC, f1, MCC))

**데이터 불러오기**

In [74]:
path = "C:/Users/32197453/Desktop/교과/4학년 2학기/기계학습이론/과제/기계학습3/"
file_list = os.listdir(path)
csv_files = []
def find_csv(files):
    for data in files:
        if '.csv' in data:
            csv_files.append(data)
find_csv(file_list)
#CSV 파일만 추출하여 목록 만들기

for i in csv_files:
    globals()[i.split('.')[0]] = pd.read_csv(f'{path}/{i}', index_col=[0])
#목록에 해당하는 CSV 파일 불러오기

In [75]:
print("Dimension of \n X_train: {1} \n Y_train: {3} \n X_test: {0} \n Y_test: {2}"
    .format(x_test.shape, x_train.shape, y_test.shape, y_train.shape))

Dimension of 
 X_train: (7162, 1119) 
 Y_train: (7162, 2) 
 X_test: (74, 1119) 
 Y_test: (74, 2)


**데이터 전처리**

In [76]:
y_train.drop('logBB', axis=1, inplace=True)
y_test.drop(y_test.columns[0], axis=1, inplace=True)
#y_train 데이터에 클래스만 남기기 위해 logBB 열을 제거함, 오류로 인해 y_test에 추가된 불필요한 열도 제거

In [77]:
col_names = list(x_train.columns)
new_col_names=[]
for i in range(1,len(col_names)+1) :
    val = "x" + f"{i}"
    new_col_names.append(val)
new_col_names[-1]
x_train.columns = new_col_names
x_test.columns = new_col_names
#RandomizedSearchCV를 하기 위해서는 [,],<가 포함된 변수명을 사용할 수 없으므로 변수명을 모두 단순하게 변경해준다.

## Random Forest 분류

**Randomized Search CV** </p>
시간의 효율을 위해 Grid Search가 아닌 Randomized Search를 사용

In [157]:
rfc = RandomForestClassifier()

rfc_param_rnd = {
    'n_estimators' : [100, 200, 400],
    'max_depth' : [4, 8, 16],
}

rfc_rnd = RandomizedSearchCV(rfc, param_distributions= rfc_param_rnd, scoring="accuracy", n_jobs= -1, verbose = 1)
rfc_rnd.fit(x_train, np.ravel(y_train))

#DataConversionWarning: A column-vector y was passed when a 1d array was expected. 오류를 해결하기 위해 y에 ravel() 함수적용

C:\Users\32197453\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 9 candidates, totalling 45 fits


RandomizedSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'max_depth': [4, 8, 16],
                                        'n_estimators': [100, 200, 400]},
                   scoring='accuracy', verbose=1)

In [158]:
print("최고 평균 정확도 : {0:.6f}".format(rfc_rnd.best_score_))
print("최고의 파라미터 : ", rfc_rnd.best_params_)

최고 평균 정확도 : 0.886214
최고의 파라미터 :  {'n_estimators': 400, 'max_depth': 16}


**탐색된 파라미터로 RandomForest 모델 적합**

In [159]:
model_rf = rfc_rnd.best_estimator_
y_pred_rfc = model_rf.predict(x_test)
print('30개의 출력값으로 간략하게 확인 \n', y_pred_rfc[:30],'\n',np.array(y_test.iloc[:,0])[:30])

30개의 출력값으로 간략하게 확인 
 [1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0] 
 [1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0]


**모델 평가 척도**

In [160]:
metrics(y_test, y_pred_rfc)

Confusion_Matrix: 
 [[30  5]
 [ 2 37]] 
 Accuracy : 0.905 
 AUC : 0.903 
 AUPRC : 0.863 
 F1-score : 0.914 
 MCC : 0.812


## XGBoost 분류

**Randomized Search CV** </p>
시간의 효율을 위해 Grid Search가 아닌 Randomized Search를 사용

In [149]:
xgb = XGBClassifier()

xgb_param_rnd = {
    'n_estimators' : [100, 200, 400],
    'learning_rate' : [0.01, 0.05, 0.1, 0.3],
    'max_depth' : [4, 8, 12],
}

xgb_rnd = RandomizedSearchCV(xgb, param_distributions= xgb_param_rnd, scoring="accuracy", n_jobs= -1, verbose = 1)
xgb_rnd.fit(x_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None...
                                           max_cat_to_onehot=None,
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           predictor=None, random_state=None, ...),
                   n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1, 0.3],
                                        'max_depth': [4, 8, 12],
                                        'n_estimators': [100, 200, 400]},
                   scoring='accuracy', verbose=1)

In [150]:
print("최고 평균 정확도 : {0:.6f}".format(xgb_rnd.best_score_))
print("최고의 파라미터 : ", xgb_rnd.best_params_)

최고 평균 정확도 : 0.886352
최고의 파라미터 :  {'n_estimators': 100, 'max_depth': 8, 'learning_rate': 0.05}


**탐색된 파라미터로 XGBoost 모델 적합**

In [151]:
model_xgb = xgb_rnd.best_estimator_
y_pred_xgb = model_xgb.predict(x_test)
print('30개의 출력값으로 간략하게 확인 \n', y_pred_xgb[:20],'\n',np.array(y_test.iloc[:,0])[:20])

30개의 출력값으로 간략하게 확인 
 [1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 0 1] 
 [1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 0 1]


**모델 평가 척도**

In [152]:
metrics(y_test, y_pred_xgb)

Confusion_Matrix: 
 [[29  6]
 [ 2 37]] 
 Accuracy : 0.892 
 AUC : 0.889 
 AUPRC : 0.843 
 F1-score : 0.902 
 MCC : 0.787


## LightGBM 분류

**Randomized Search CV** </p>
시간의 효율을 위해 Grid Search가 아닌 Randomized Search를 사용

In [153]:
lgbm = LGBMClassifier()


lgbm_param_rnd = {
    'n_estimators' : [100, 200, 400],
    'learning_rate' : [0.01, 0.05, 0.1, 0.3],
    'max_depth' : [4, 8, 12],
}

lgbm_rnd = RandomizedSearchCV(lgbm, param_distributions= lgbm_param_rnd, scoring="accuracy", n_jobs= -1, verbose = 1)
lgbm_rnd.fit(x_train, np.ravel(y_train))



Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(estimator=LGBMClassifier(), n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1, 0.3],
                                        'max_depth': [4, 8, 12],
                                        'n_estimators': [100, 200, 400]},
                   scoring='accuracy', verbose=1)

In [154]:
print("최고 평균 정확도 : {0:.6f}".format(lgbm_rnd.best_score_))
print("최고의 파라미터 : ", lgbm_rnd.best_params_)

최고 평균 정확도 : 0.886075
최고의 파라미터 :  {'n_estimators': 200, 'max_depth': 12, 'learning_rate': 0.01}


**탐색된 파라미터로 LightGBM 모델 적합**

In [155]:
model_lgbm = lgbm_rnd.best_estimator_
y_pred_lgbm = model_lgbm.predict(x_test)
print('30개의 출력값으로 간략하게 확인 \n', y_pred_lgbm[:20],'\n',np.array(y_test.iloc[:,0])[:20])

30개의 출력값으로 간략하게 확인 
 [1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0 1] 
 [1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 0 1]


**모델 평가 척도**

In [156]:
metrics(y_test, y_pred_lgbm)

Confusion_Matrix: 
 [[23 12]
 [ 1 38]] 
 Accuracy : 0.824 
 AUC : 0.816 
 AUPRC : 0.754 
 F1-score : 0.854 
 MCC : 0.674


## 평가
이 데이터에 대해서 5가지 모든 지표에서 RandomForest, XGBookst, LightGBM 순으로 예측 성능이 더 높은 값을 보이고 있다. RandomForest가 XGBoost보다 높은 이유는 GridSearch가 아닌 RandomizedSearch를 사용하고 더 다양한 값을 넣지 않았기 때문에 XGBoost의 파라미터가 충분히 고려되지 않아 과적합이 발생했기 때문이라고 예상된다. 그리고 이 둘의 차이는 크지 않지만 이 둘에 비해서 LightGBM은 더 낮은 예측 성능을 보이고 있다. Test 데이터의 인스턴스 개수는 74개로 LightGBM은 10,000개 미만의 데이터에 대해선 과적합이 쉽게 발생하기 때문에 이런 결과가 나온 것이라고 추측할 수 있다.